In [16]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [35]:
mnist_dataset,mnist_info=tfds.load(name='mnist',as_supervised=True,with_info=True)

minst_train=mnist_dataset['train']
mnist_test=mnist_dataset['test']

num_validation_samples=0.1*mnist_info.splits['test'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples

def scale(image,label):
    image=tf.cast(image,tf.float64)
    image /= 255.
    return image, label

scaled_train_validation_data=mnist_train.map(scale)

scaled_test_data=mnist_test.map(scale)

BUFFER_SIZE=10000

shuffled_train_validation=scaled_train_validation_data.shuffle(BUFFER_SIZE)

validation_data=shuffled_train_validation.take(num_validation_samples)
train_data=shuffled_train_validation.skip(num_validation_samples)

BATCH_SIZE = 100

train_data=train_data.batch(BATCH_SIZE)

##Giving the total num of validation data to take it as whole instead of batching
validation_data=validation_data.batch(num_validation_samples)

test_data=scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

2022-10-29 18:05:59.854006: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [52]:
input_layer=28*28
hidden_layer=100
output_layer=10

model=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer,activation='relu'),
    tf.keras.layers.Dense(hidden_layer,activation='tanh'),
    tf.keras.layers.Dense(output_layer,activation='softmax')
])


### Choose the optimizer and Loss Function

In [53]:
adam=tf.keras.optimizers.Adam(learning_rate=0.002)
model.compile(optimizer=adam,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

## Training

In [54]:
NUM_EPOCHS = 5

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
590/590 - 5s - loss: 0.2431 - accuracy: 0.9284 - val_loss: 0.1164 - val_accuracy: 0.9710 - 5s/epoch - 9ms/step
Epoch 2/5
590/590 - 3s - loss: 0.1031 - accuracy: 0.9683 - val_loss: 0.1044 - val_accuracy: 0.9710 - 3s/epoch - 5ms/step
Epoch 3/5
590/590 - 3s - loss: 0.0716 - accuracy: 0.9772 - val_loss: 0.0589 - val_accuracy: 0.9850 - 3s/epoch - 5ms/step
Epoch 4/5
590/590 - 5s - loss: 0.0542 - accuracy: 0.9830 - val_loss: 0.0509 - val_accuracy: 0.9840 - 5s/epoch - 9ms/step
Epoch 5/5
590/590 - 12s - loss: 0.0436 - accuracy: 0.9863 - val_loss: 0.0374 - val_accuracy: 0.9850 - 12s/epoch - 21ms/step


In [55]:
test_loss,test_accuracy=model.evaluate(test_data)

1/1 [==============================] - 0s 245ms/step - loss: 0.0688 - accuracy: 0.9794


2022-10-29 18:47:09.159104: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 62720000 exceeds 10% of free system memory.
